# Reinforcement Learning Based Cognitive Radio Project

*by Leif Skunberg, Michael Lyons, and Charles Culpepper, December, 2017*

## Table of Contents
* [Introduction](#Introduction)
* [Problem](#Problem)
* [Methods](#Methods)
* [Team](#Team)
* [Results](#Results)
* [Conclusions](#Conclusions)

## Introduction

#### Cognitive Radio

Radio is ubiquitous in our society. Radio waves cover our globe, and provide everything from standard communication and music in your car to the frequencies used to provide the world with wireless internet. Radio waves are a type of electromagnetic radiation at a given range of frequencies. There are other waves in this spectrum, depending on the frequency. These include everything from x-rays and infrared to UV and standard light. Currently, you tune a radio to a frequency, and it will pick up any communications on that channel. The stronger your radio, the more of the range it can access and transmit through. Some frequencies are off limits for civilians, and some others are used for specific services, like wi-fi. But within the standard range, most people know what it means to tune a radio to a frequency – the number on your favorite radio station.

In addition to simply tuning into the right frequency, there are several components to radio communications, the most obvious of which is interference. As you drive out of your local area, you may notice your radio station start to pick up another, or maybe die off into static. Either way, you’re seeing a form of interference. Either way, you’re seeing a form of interference or loss of signal propagation.  Ideally, if you can stay on the channels with the least interference, you’ll always have a clear line of communication. 

A cognitive radio is one that can see, and over time, learn, which frequencies have the least interference at a given time, and can hop to the channel with least interference so that the communication stays clear. Basically, a cognitive radio is one that switches frequencies for you, depending on what it thinks is the clearest line of communication. At it’s most basic, it can simply decide between lines that are open, and lines that are already in use, keeping the user on the open lines as much as possible. This frequency hopping is called Frequency Hopping Spread Spectrum (FHSS) [Ren].

The idea for a cognitive radio started back in 1998, where Joseph Mitola III made a proposal for it at a seminar for KTH (translates to Royal Institute of Technology in Stockholm). There is no fully working example of a live cognitive radio to date. Rather, it is an established concept that is accepted as possible but is still a goal that the industry is actively working toward.
The latest trends in AI are what makes advances in cognitive radio possible. For a radio receiver to know which lines are open (or which have a satisfactory level of interference), it needs to know, compute, remember, and learn about the elements included therein. This necessitates some form of machine learning. As our technology develops, we are getting closer to processing the needed calculations in real time, meaning, as the communication happens.
Currently, most experiments regarding cognitive radio take place either with sample data or with captured data from a live radio. While many have explored both in depth, no one has been able to perform the functionality as the communication happens.

#### Cognitive Radio Experiment in Reinforcement Learning

These same limitations impact our own experiment. Due to time constraints, we chose to run our experiment with simple data. The idea is that you should be able to apply reinforcement learning to FHSS using GNURadio, creating a software-based cognitive radio. The intent is also to encrypt the message, and so a final version would be able to send end-to-end secure communications, over a radio, on the clearest lines available, with no message deterioration. While we were unable to perform the functions on live data, we created sample data and a reinforcement learning algorithm in order to make a proof of concept. We would have liked to include RSA encryption code, but as is it would just be tacked on without any clear tie-in.

#### RSA Encryption

There are many ways to encrypt and decrypt data for preparation to transmit it in a radio signal. We chose an asymmetric form named RSA. The basic functionality of an asymmetric cryptographic system is the existence of two keys, a public key, and a private key. It makes more intuitive sense if you consider the public key a lock. Anyone can lock something with it, but only the owner of the private key can unlock those locks. 

A user would typically use the public key of their planned recipient to encrypt some data.That data can then be encapsulated in another encryption, created by the user’s own public key. Anyone can unlock the outer layer of encryption because it uses a public key, but the inner one still needs the corresponding private key. The point of this is to prove that the sender is authentic. That user’s public key, while public, is unique, and proves that it was encrypted by the original sender. The recipient then receives the communication, uses the public key to decrypt the outer layer, and his own private key to decrypt the inner layer, and get the data. That ensures that the sender is authentic, the message is secure, and the receiver is the intended person. This system provides end to end security for any data that can be stored and transmitted digitally [Milanov].

## Problem

![problem figure](problem.png)
(Figure 1)

According to ABI research, as of 2013 there was over ten billion wireless devices on earth. By 2020, there will be an expected amount of over 30 billion. This presents in interesting challenge of spectrum congestion by devices of all types. Spectrum congestion has shown that our wireless capabilities can be hindered and limited with interference from competing signals. In this project we asked ourselves if we could use our knowledge from our CS440 class to find possible solutions to this problem. By using reinforcement learning algorithms, could hopping to channels that are less congested be the solution to spectrum scarcity?

Figure 1 gives a graphical high level overview of how our algorithm works with the data provided. The channel with the lowest interference at an given time is chosen. The goal is to find patterns in data that will allow a radio to choose the best possible channels at a given location and time. Peak congestion on one channel at a given time (ie Netflix being streamed after dinner) may be different from what is available at another time (ie A drone streaming a live feed to the internet). After a prolonged period of time, preferences for channels that are known to have less interference during specific times gives an edge to data transmissions that would otherwise be hindered by interference and congestion.

## Methods

Early on in the project, test data was gathered using a BladeRF x40 software defined radio. Ten channels on the 2.4 GHz spectrum was captured over a period of 15 seconds ten times. Original data that was gathered for testing was over *130 Gigabytes* in size, making it very difficult to parse. A representation of the data created was needed, as actual data created by the radio was infeasible to aggregate. Representative values were made for these data points that have the most weight on radio transmissions were used for our reinforcement learning algorithm [ABI Research]. 

The original test data gave us a good idea on how to simulate test data without creating many hundred gigabytes of data. The original files were captured in binary format(SC16 Q11). Below are graphs that were plotted in GNU Octave of the captured data. 

![no activity](noactivity.jpeg)
Figure 2. This graph represents an empty channel that has no data being transmitted over a 15 second period on a home wifi network. 

![activity](activity.jpeg)
Figure 3. This graph represents a channel that has multiple active transmissions being sent and received by devices in proximity of the home wifi network.



### The Algorithm

Our reinforcement learning algorithm follows roughly the same structure as the algorithm outlined in the lecture notes.
	
	for transmission time
		next_time = time + 1
		if next_time < transmission time
			initialize_Q_table(time)
			move = get_move(time)
			update_reinforcement(time)

The key difference between a reinforcement algorithm for a maze or a game like tic-tac-toe and our simulated radio transmission is that reinforcement happens at a terminal state, like reaching the end of the maze or ending a game of tic-tac-toe. However, in our simulated radio transmission, where the move from the current state to the next state is independent, reinforcement has to happen at every hop. Our reinforcement function is simple:

if  move is optimal
	Q(state, move) = Q(state, move) * (1 + (1 - [learning rate]))
else 
	Q(state, move) = Q(state, move) * [learning rate]

In other words, if a move is optimal (it has the lowest interference), then it is increased by, say, 5%. If it is not optimal, then it is decreased by the learning rate. Ideally, based upon the assumption that there would be a strong correlation between datasets generated from a pattern, the algorithm could exploit historical knowledge about previous datasets to quickly hop frequencies.

### The Original Model

Our original model has two entities, the template and the data-set. 

The template represents a location with patterns in transmission at given times of the day. For example, a channel c might have, on average and over the long term, heavy interference at time t, while the dataset is essentially an instantiation of the template. 

This model is essentially a matrix with rows representing time snapshots and columns representing different channels. The template goes through and fills the matrix with several variables.

	alpha: percent chance to lower interference by x
	x: value to lower interference 

In addition to those two, an empty channel will have a length that propagates down the transmission matrix, with future channels in the same transmission having similar interferences. A transmission is a set of time snapshots in a column of the matrix.

When the template is instantiated into a dataset, all of the channels in all of the cells of the matrix have their interference set. A transmission’s channels throughout time will have similar interferences.

### Original Model Fails

We’ve included a discussion of the original model because it highlights our original thinking and the process, and pitfalls, of trying to create a sufficient model. 

As it turns out, the glaring problem with the original model only became apparent upon testing. After much testing, our trained Q table was only selecting optimal channels roughly 10% of the time from newly instantiated testing data sets. Confused and desperately looking for serious bugs, it finally occurred to us that we had only ten channels, and 10% is exactly what you would expect if you simply picked a channel by random! 

So what went wrong?

It turns out the original model for data generation created data sets that were too different from one another. Consider channel c1, which has an alpha of 17%. The probability that c1 will have the same value from one dataset to another is just 0.17 * 0.17 = 0.0289 or just 2.89%!

### The New Model

Our current model is much more simple. The template generates the interference values for every channel using the alpha-x system, but keeps it constant across all future datasets generated from the template. Generation of a dataset involves applying a noise function to every channel in the matrix. 

## Team

### Charles
For this project I wrote the encryption code using the included library, tested the algorithm code, and helped research and develop our "cognitive radio" concept.

### Leif
I did the project idea, bug hunting, radio, BladeRF data collection, problem statement and top block.

### Michael
I wrote the algorithm and helped with the model for our data generation.

## Results

In [15]:
from Channel import *
from Experiment import *
from ReinforcementLearning import *

### Experiment
There are eight parts to an experiment as defined here:
* Transmission length: The time frame or simply the rows in the matrix.
* Channel length: The number of channels or columns in the matrix.
* Epsilon: This number controls how often a random move is selected in training. If it is 0.5, then 50% of the time a random move will be selected.
* Epsilon Decay Rate: The factor that epsilon decays by each iteration, so that it will tend towards zero. If epsilon is 80% and the epsilon decay rate is 30%, then epsilon will be 24% after one iteration.
* Training iterations: The number of times the Q table is trained on *one* data set.
* Learning rate: The rate at which reinforcements are decreased or increased.
* Train runs: The number of times the Q table is trained on *different* data sets.
* Test runs: The number of different times the Q table is trained on *different* data sets.

#### To Run
Feel free to fiddle around with various variables. There isn't too much surprise here, since it's a fairly simple model most of the time it will be in the high 95% correct. If any of the variables are too high, it will simply take took long to complete. Transmissions length, channel length, training iterations, train runs, and test runs should all be kept under 10,000 as an upper-bound.

One thing to note: A very wide, that is a lot of channels but not a huge transmission length, is pretty hard to predict. This is likely because there are so many channels near the optimal channel that the slight variation from dataset to dataset is enough to cause the algorithm to pick the wrong channel.

#### Small matrix

In [16]:
transmission_length = 10
channel_length = 5
epsilon = 0.999
epsilon_decay_rate = 0.999
training_iterations = 1000
learning_rate = 0.98
train_runs = 10
test_runs = 100

performance = run_experiment(transmission_length, channel_length, epsilon, epsilon_decay_rate, training_iterations, learning_rate, train_runs, test_runs)
print("With " + str(train_runs) + " training runs and " + str(test_runs) +" test runs, on average " + format_double(performance * 100) + "% of optimal channels were selected.")

Done Training
With 10 training runs and 100 test runs, on average 100.00% of optimal channels were selected.


#### Square Matrix

In [17]:
transmission_length = 100
channel_length = 100
epsilon = 0.999
epsilon_decay_rate = 0.999
training_iterations = 100
learning_rate = 0.98
train_runs = 10
test_runs = 100

performance = run_experiment(transmission_length, channel_length, epsilon, epsilon_decay_rate, training_iterations, learning_rate, train_runs, test_runs)
print("With " + str(train_runs) + " training runs and " + str(test_runs) +" test runs, on average " + format_double(performance * 100) + "% of optimal channels were selected.")

Done Training
With 10 training runs and 100 test runs, on average 93.14% of optimal channels were selected.


#### Long Matrix

In [18]:
transmission_length = 1000
channel_length = 10
epsilon = 0.999
epsilon_decay_rate = 0.999
training_iterations = 100
learning_rate = 0.98
train_runs = 5
test_runs = 100

performance = run_experiment(transmission_length, channel_length, epsilon, epsilon_decay_rate, training_iterations, learning_rate, train_runs, test_runs)
print("With " + str(train_runs) + " training runs and " + str(test_runs) +" test runs, on average " + format_double(performance * 100) + "% of optimal channels were selected.")

Done Training
With 5 training runs and 100 test runs, on average 94.58% of optimal channels were selected.


#### Wide Matrix

In [19]:
transmission_length = 10
channel_length = 1000
epsilon = 0.999
epsilon_decay_rate = 0.999
training_iterations = 10
learning_rate = 0.98
train_runs = 10
test_runs = 10

performance = run_experiment(transmission_length, channel_length, epsilon, epsilon_decay_rate, training_iterations, learning_rate, train_runs, test_runs)
print("With " + str(train_runs) + " training runs and " + str(test_runs) +" test runs, on average " + format_double(performance * 100) + "% of optimal channels were selected.")

Done Training
With 10 training runs and 10 test runs, on average 20.00% of optimal channels were selected.


#### Comparison to Greedy Algorithm

A question that was asked by our team early on was how would this algorithm give an advantage to a wireless device that would otherwise use a greedy algorithm to always choose the less congested channel? Well time and device resources must be taken to scan the spectrum for a preferred channel. By having patterns in the signals over a period of time, one can immediately know which channel would be preferred over another. Obviously in our setup a greedy algorithm would be *much* better than our loop-loving reinforcement algorithm, but in the real-world, where transmission data would be much larger, having a Q-table that instantly gives you a *pretty good* guess is much better than a slow greedy algorithm scanning channel after channel.

## Conclusions

To be candid, our project did not turn out how we had hoped. Initially we wanted to use real radios and hop real channels with a well honed artifical intelligence, but it didn't work out that way. For various reasons, we had to scrap real world data. And then, when it became clear that the data we were generating was essentially random noise, we had to switch to a much more simple model. However, because of what we have learned about reinforcement, radios, encryption, and simple models, we don't consider it a failure in the slightest. 



#### What Michael Learned
In adapting the reinforcement algorithm we learned in class to this specific problem, I learned a lot more about the general idea behind reinforcement. Real world systems don't always have "terminal" states to use as reinforcements, nor do they generlize as nicely as simple games. After this class, I plan to try (and probably fail) using these same techniques with Monopoly!

As far as modeling goes, I would have never guessed it was so difficult to simulate one variable, interference, in such a way that allows a pattern to develop from dataset to dataset. If I had to do it again, I would likely start from scratch and try and include more in the state (such as day, month, year, etc) and get it closer to looking like real life. 

#### What Leif Learned
This project not only helped me understand the intricacies of the wireless spectrum, but how reinforcement learning techniques can be applied to help solve unique problems we will be faced with in the future. Another take away from this project was how complicated wireless data transmission can be. The raw data we gathered originally during the project faced us with a congestion problem that was not considered when we started. Adapting to these challenges on the fly made the project challenging and interesting.

#### What Charles Learned
While I've used various forms of technology throughout my life, I never really delved deep into radio technology and how it worked. With this project I did a lot fo research into radios and their intricacies. I learned quite a bit about the electreomagnetic spectrum and how we can write software to leverage parts of the spectrum for our benefit.  Past that, the concept of apply AI to a frequency hopping radio was a unique way to apply what we learned and class, and expanded my understanding of practical applications for machine learning.

### References

* [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu/), Publisher. 2014.
* [Lecture notes algorithm is based off of](https://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/14%20Introduction%20to%20Reinforcement%20Learning.ipynb)
* [ABI Research](https://www.abiresearch.com/market-research/service/m2m-iot-ioe/)
* [Ren](http://researcharchive.vuw.ac.nz/xmlui/bitstream/handle/10063/1976/thesis.pdf?sequence=2)
* [Milanov](https://sites.math.washington.edu/~morrow/336_09/papers/Yevgeny.pdf)